## CSD Decomposition

In this unit, we first CSD decompose huge Unitaries into block multiplication of small ones.

We do this because the SK-algorithm we have below is inefficient for large unitaries.

In [1]:
import numpy as np
from scipy.linalg import cossin, norm
from functools import lru_cache
import pickle

d = 4 # corresponds to 8-qubit unitary

# ——————————————————————————————
# 1) Load or construct your 256×256 unitary U
# ——————————————————————————————
# For example, a random Haar unitary (for testing only):
def random_unitary(d):
    A = (np.random.randn(d, d) + 1j*np.random.randn(d, d)) / np.sqrt(2)
    # QR decomposition, then fix det=1
    Q, R = np.linalg.qr(A)
    D = np.diag(np.exp(-1j * np.angle(np.diag(R))))
    return Q @ D

def to_su2(m):
    d = m.shape[0]
    sign, logdet = np.linalg.slogdet(m)
    φ = np.angle(sign*np.exp(logdet))
    return m * np.exp(-1j*φ/(d))

def random_su2(d):
    U = random_unitary(d)    # gives U(2)
    return to_su2(U)         # now in SU(2)


U = random_su2(d)
# Verify unitarity (should be ≲1e-12)
print("Unitary check:", norm(U.conj().T @ U - np.eye(d)))

# ——————————————————————————————
# 2) Perform CSD with p = 128, q = 128
# ——————————————————————————————
p = 2
U_blk, CS_blk, Vh_blk = cossin(U, p = p, q = p)

# U_blk  = [[ u1,  0 ],[ 0,  u2 ]]       shape (256×256)
# CS_blk = [[ C,  -S ],[ S,   C ]]       shape (256×256)
# Vh_blk = [[v1h,  0 ],[ 0, v2h ]]       shape (256×256)

# Then
C =    CS_blk[:p,   :p]
S = - CS_blk[:p, p:]   # mind the minus sign if you want S (the factorization uses –S in the top right)
U1      =    U_blk[:p,   :p]
U2      =    U_blk[p:, p:]
V1h     =   Vh_blk[:p,   :p]
V2h     =   Vh_blk[p:, p:]

# Shapes:
# U1, U2, V1h, V2h are each (128×128)
# C, S are diagonal blocks of shape (128×128)

print("U1 shape:", U1.shape)
print("U2 shape:", U2.shape)
print("C diag elements (first 5):", np.diag(C)[:5])
print("S diag elements (first 5):", np.diag(S)[:5])

# ——————————————————————————————
# 3) (Optional) Reconstruct U to check exactness
# ——————————————————————————————
upper = np.block([
    [ U1,              np.zeros((p,p), dtype=complex) ],
    [ np.zeros((p,p), dtype=complex),   U2           ]
])
middle = np.block([
    [  C, -S ],
    [  S,  C ]
])
lower = np.block([
    [ V1h,             np.zeros((p,p), dtype=complex) ],
    [ np.zeros((p,p), dtype=complex),   V2h           ]
])
U_rec = upper @ middle @ lower
print("Reconstruction error:", norm(U - U_rec))

# ——————————————————————————————
# 4) What you get
# ——————————————————————————————
#   • U1, U2 ∈ U(128)
#   • V1, V2 ∈ U(128) via V1 = V1h.conj().T, V2 = V2h.conj().T
#   • C = diag(cos θ_i), S = diag(sin θ_i)

# Next steps (quantum circuit):
#  - implement V0†/V1† as an 8-qubit multiplexor controlled on the high bit,
#  - implement the block-diagonal [C, –S; S, C] via 128 controlled Ry(2θ_i) on the low 7 bits,
#  - implement X0/X1 = U1/U2 at the end, again as a multiplexor.


Unitary check: 6.5863774786378015e-16
U1 shape: (2, 2)
U2 shape: (2, 2)
C diag elements (first 5): [0.82216376 0.55662177]
S diag elements (first 5): [0.56925104 0.83076604]
Reconstruction error: 9.293067637693925e-16
